In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json

In [ ]:
# main page with letters

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


In [26]:
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])

In [12]:
lista_personaggi = pd.read_csv("./Datasets/list_of_characters_with_biography.csv")
lista_personaggi.head()

,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [16]:
lista_personaggi.head()


,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [8]:
lista_personaggi.shape

(3747, 2)

In [3]:
def cerca_bio(page_tree):
    biography_list = []
    biography_initial = page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    return biography_list

In [23]:
def cerca_serie(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "tv series":
            tag_lista_film = element

    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [22]:
def cerca_film(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "movie":
            tag_lista_film = element
    
    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [13]:
def cerca_relazione_recursive(list_item):
  if not list_item:
    return []
  next_items = list_item.find_all("li")
  if not next_items:
    return [list_item.text.split(" - ",1)]
  else:
    recursive_list = []
    for item in next_items:
      recursive_list += cerca_relazione_recursive(item)
    return recursive_list



In [14]:
def cerca_relazione(page_tree):
  relationship_initial_tag = page_tree.find(id="Relationships")
  if relationship_initial_tag:
    rel_table = relationship_initial_tag.find_next("table")
    relation_list = []
    if rel_table:
      for ul_list in rel_table.find_all("ul"):
        for list_item in ul_list.find_all("li"):
          relation_list += cerca_relazione_recursive(list_item)
    return relation_list


In [24]:
from IPython.display import clear_output
lista_personaggi = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv")
#lista_personaggi["Biografia"] = None
lista_personaggi["Film"] = None
lista_personaggi["Serie"] = None
# lista_personaggi["Relazioni"] = None

num_iterazioni = 0

for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    character_page_response = requests.get(row["Link"])
    character_page_tree = BeautifulSoup(character_page_response.text)
    #biografia = cerca_bio(character_page_tree)
    film = cerca_film(character_page_tree)
    serie = cerca_serie(character_page_tree)
    # relazioni = cerca_relazione(character_page_tree)
    #lista_personaggi.at[idx, "Biografia"] = biografia
    lista_personaggi.at[idx, "Film"] = json.dumps(film)
    lista_personaggi.at[idx, "Serie"] = json.dumps(serie)
    # lista_personaggi.at[idx, "Relazioni"] = json.dumps(relazioni)


100%|██████████| 3746/3746 [11:27<00:00,  5.45it/s]


In [25]:
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo.csv", index=False)

In [37]:
lista_personaggi = pd.read_csv("./Datasets/scraping_personaggi_completo.csv")
lista_personaggi.head(20)
filt = lista_personaggi.query("Nome == 'Grandmother'")
print(len(filt["Relazioni"]))


0


In [38]:
lista_personaggi["Processed_Name"] = None
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  lista_personaggi.at[idx, "Processed_Name"]= row["Nome"].replace("'", " ").replace('"', " ").replace(" \u2020","")
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_name.csv", index=False)


100%|██████████| 3746/3746 [00:00<00:00, 21648.21it/s]


In [39]:
test =[['test','test']]
print(json.dumps(test))
lista_personaggi = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv")
lista_personaggi["Filtered_relazioni"] = None
errors =0
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  try:
    relations = json.loads(row["Relazioni"])
    filtered_relations = []
    for relation in relations:
        querystring = "Processed_Name == '%s'" % (
          relation[0].replace("'", " ").replace('"', " ").replace(" \u2020",""))
        if len(lista_personaggi.query(querystring)) != 0:
          filtered_relations.append(relation)

    lista_personaggi.at[idx, "Filtered_relazioni"] = filtered_relations
  except Exception as e:
    errors+=1
print("Errors %s"%(errors))
  
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_relations.csv", index=False)


[["test", "test"]]


100%|██████████| 3746/3746 [00:47<00:00, 79.47it/s] 


Errors 171


In [56]:
def get_film_info(page_tree):
  try:
    side_info = page_tree.find(
        "aside", class_="portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default")
    divs = side_info.find_all(
        "div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    directors_list = []
    writer_list = []
    producer_list = []
    composer_list = []
    release_date = None
    runtime = None
    boxoffice = None
    for div in divs:
        if div["data-source"] == "director":
          directors_tags = div.find_all("a")
          for director in directors_tags:
            if director["href"] != "#cite_note-1":
              directors_list.append([director.text, director["href"]])
        elif div["data-source"] == "writer":
          writers_tags = div.find_all("a")
          for writer in writers_tags:
            if writer["href"] != "#cite_note-1":
              writer_list.append([writer.text, writer["href"]])
        elif div["data-source"] == "producer":
          producer_tags = div.find_all("a")
          for producer in producer_tags:
            if producer["href"] != "#cite_note-1":
              producer_list.append([producer.text, producer["href"]])
        elif div["data-source"] == "composer":
          composer_tags = div.find_all("a")
          for composer in composer_tags:
            if composer["href"] != "#cite_note-1":
              composer_list.append([composer.text, composer["href"]])
        elif div["data-source"] == "release":
          tags = div.find_all("div")
          for tag in tags:
            release_date = tag.text
        elif div["data-source"] == "runtime":
          tags = div.find_all("div")
          for tag in tags:
            runtime = tag.text
        elif div["data-source"] == "boxoffice":
          tags = div.find_all("div")
          for tag in tags:
            boxoffice = tag.text
    return ([directors_list, writer_list, producer_list, composer_list, release_date, runtime, boxoffice])
  except Exception as e:
    print(e)

  

def get_synopsis(page_tree):
  try:
    synopsis = []
    title = page_tree.find(
        id="Synopsis")
    next_tag = title.find_next()
    while next_tag.name == "p" and next_tag:
      synopsis.append(next_tag.text)
      next_tag = next_tag.find_next_sibling()
    return (synopsis)
  except Exception as e:
    print(e)

  

def film_scraping(film_name="Avengers: Infinity War", film_url="https://marvelcinematicuniverse.fandom.com/wiki/Avengers:_Infinity_War"):
  try:
    film_page_response = requests.get(film_url)
    film_page_tree = BeautifulSoup(film_page_response.text)
    # film_info = get_film_info(film_page_tree)
    get_synopsis(film_page_tree)
  except Exception as e:
    print(e)


In [59]:
lista_personaggi = pd.read_csv("Datasets/scraping_personaggi_completo_filtered_relations.csv")
lista_film_links = []
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  films = json.loads(row["Film"])
  for film in films:
    if film[1] not in lista_film_links:
      lista_film_links.append(film[1])
print(lista_film_links)


100%|██████████| 3746/3746 [00:00<00:00, 10579.04it/s]

['/wiki/Avengers:_Infinity_War', '/wiki/Captain_America:_The_Winter_Soldier', '/wiki/Spider-Man:_Homecoming', '/wiki/Guardians_of_the_Galaxy_Vol._2', '/wiki/The_Incredible_Hulk', '/wiki/Shang-Chi_and_the_Legend_of_the_Ten_Rings', '/wiki/Captain_America:_The_First_Avenger', '/wiki/The_Avengers', '/wiki/Avengers:_Age_of_Ultron', '/wiki/Spider-Man:_No_Way_Home', '/wiki/Iron_Man_(film)', '/wiki/Captain_Marvel_(film)', '/wiki/Iron_Man_2', '/wiki/Guardians_of_the_Galaxy_Vol._3', '/wiki/Doctor_Strange_(film)', '/wiki/Avengers:_Endgame', '/wiki/Eternals_(film)', '/wiki/Iron_Man_3', '/wiki/Black_Widow_(film)', '/wiki/The_Marvels', '/wiki/Ant-Man_and_the_Wasp', '/wiki/Black_Panther:_Wakanda_Forever', '/wiki/Thor_(film)', '/wiki/Ant-Man_(film)', '/wiki/Captain_America:_Civil_War', '/wiki/Spider-Man:_Far_From_Home', '/wiki/Ant-Man_and_the_Wasp:_Quantumania', '/wiki/Thor:_Love_and_Thunder', '/wiki/Thor:_Ragnarok', '/wiki/Black_Panther_(film)', '/wiki/Guardians_of_the_Galaxy_(film)', '/wiki/Blade_(f

In [ ]:
lista_personaggi.head(20)